### MILP Formulation

Variables definition:

$S$: The set of all possible sites

$P$: The set of production sites

$D$: The set of distribution sites

$d_s$: 1 if site s is a distribution center and 0 otherwise

$p_s$: 1 if site s is a standard production center and 0 otherwise

$a_s$: 1 if site s is an automatic production center and 0 otherwise

$X_{si}$: Client-site incidence matrix, which associates each client to a site, 1 if connected, 0 otherwise

$X_{ss'}$: Site-site incidence matrix, which associates each distribution center to a production center, 1 if connected, 0 otherwise

$\delta_i$: The demand of client i

Cost functions:
- Building costs:
$$
BC(x) = \sum_{s \in S} c_D^b d_s + c_P^b p_s + c_A^b a_s
$$
- Production costs:
$$
PC(x) = \sum_{i \in I} \left[ \sum_{s \in S} X_{is}\delta_i \Big( c_P^pp_s + (c_P^p-c_A^p)a_s + (c_P^p-\sum_{s' \in S} \left(X_{ss'}c_A^pa_{s'}\right) + c_D^p)d_s \Big)\right]
$$
- Routing costs
$$
RC(x) = \sum_{i \in I} \left[ \sum_{s \in S} X_{is}\delta_i \Big( c_2^r\Delta(s,i) + \sum_{s' \in S}X_{ss'}c_1^r\Delta(s,s')d_s \Big) \right]
$$
- Capacity costs
$$
CC(x) = \sum_{s \in S} c^u \left[ p_s slack_s^p +a_s slack_s^a\right]
$$

- Total costs
$$
TC(x) = BC(x) + PC(x) + RC(x) + CC(x)
$$

Subject to:
\begin{align*}
    &a_s, d_s, p_s \in \{0,1\} \\
    &0 \leq p_s+a_s+d_s \leq 1 \quad \forall s \in S\\
    &p_s+a_s+d_s \geq X_{si} \quad \forall s \in S, \forall i \in I\\
    &p_s+a_s \geq X_{ss'} \quad \forall s \in P, \forall s' \in D\\
    &\sum_{s \in [P \cup D]} X_{si}= 1 \quad \forall i \in I\\
    &\sum_{s \in P} X_{ss'}= 1 \quad \forall s' \in D\\
    &slack_s^p, slack_s^a \geq 0 \\
    &slack_s^p \geq \sum_{i \in I} \left[ \delta_i \left( X_{is}+\sum_{s' \in S} X_{is'} X_{ss'} \right) -(u_P) \right] \forall s \in S\\\\
    &slack_s^a \geq \sum_{i \in I} \left[ \delta_i \left(X_{is}+\sum_{s' \in S}X_{is'} X_{ss'} \right) -(u_P+u_A)   \right] \forall s \in S\\\\
\end{align*}

We now try to solve the MILP with the Gurobi solver

We import the necessary libraries and we check the working directory

In [67]:
import json
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

print(os.getcwd())

c:\Users\marco\Documents\Università\ENPC\REOP\Project 2021


We create a function that reads the json file and assigns the data to known variables 

In [68]:
def read_json(name):
    with open("sujet\KIRO-" + name + ".json") as file:
        data = json.load(file)
        
    clients = pd.json_normalize(data, record_path=['clients'])
    sites = pd.json_normalize(data, record_path=['sites'])

    buildingCosts = pd.DataFrame(data['parameters']['buildingCosts'],index=[0])
    productionCosts = pd.DataFrame(data['parameters']['productionCosts'],index=[0])    
    routingCosts = pd.DataFrame(data['parameters']['routingCosts'],index=[0])

    capacities = pd.DataFrame(data['parameters']['capacities'],index=[0])
    capacities['capacityCost'] = data['parameters']['capacityCost']

    ssdistance = pd.json_normalize(data, record_path=['siteSiteDistances'])

    scdistance = pd.json_normalize(data, record_path=['siteClientDistances'])
    return clients, sites, buildingCosts, productionCosts, routingCosts, capacities, ssdistance, scdistance


clients, sites, buildingCosts, productionCosts, routingCosts, capacities, ssdistance, scdistance = read_json("small")

In [ ]:
possible_sites = [i for i in range(1,sites.loc[:,'id'].size+1)]
all_clients = [i for i in range(1,clients.loc[:,'id'].size+1)]
xsi = {(i,j) for i in possible_sites for j in all_clients}
xss = {(i,j) for i in possible_sites for j in possible_sites if i != j}
demands = {i:clients.loc[i-1,'demand'] for i in all_clients}
all_ssdistance = {(i,j):ssdistance.iloc[i,j] for i in range(sites.loc[:,'id'].size) for j in range(sites.loc[:,'id'].size) if i != j}
all_scdistance = {(i,j):scdistance.iloc[i,j] for i in range(sites.loc[:,'id'].size) for j in range(clients.loc[:,'id'].size)}

mdl = gp.Model("Air Liquide")

p_s = mdl.addVars(possible_sites, name="p_s", vtype=GRB.BINARY)
a_s = mdl.addVars(possible_sites, name="a_s", vtype=GRB.BINARY)
d_s = mdl.addVars(possible_sites, name="d_s", vtype=GRB.BINARY)
x_si = mdl.addVars(xsi, name="x_si", vtype=GRB.BINARY)
x_ss = mdl.addVars(xss, name="x_ss", vtype=GRB.BINARY)
slack_p = mdl.addVars(possible_sites, name="slack_p", vtype=GRB.CONTINUOUS)
slack_a = mdl.addVars(possible_sites, name="slack_a", vtype=GRB.CONTINUOUS)

print(x_ss)

We now define the cost functions

In [125]:
# I was trying to avoid the limitation of no more than 2 multiplications imposed by Gurobi
cost1 = []
for i in range(1,sites.loc[:,'id'].size+1):
    for j in range(1,sites.loc[:,'id'].size+1):
        if i!=j:
            mult0 = x_ss[i,j]*productionCosts.loc[0,'automationBonus']
            mult00 = mult0*a_s[j]
            cost1.append(mult00)
cost11 = {i:cost1[i] for i in possible_sites}

cost2 = []
for i in range(1,sites.loc[:,'id'].size+1):
    for j in range(1,sites.loc[:,'id'].size+1):
        if i!=j:
            mult0 = x_ss[i,j]*routingCosts.loc[0,'primary']
            mult00 = mult0*d_s[j]
            cost2.append(mult00)
cost22 = {i:cost2[i] for i in possible_sites}

def prod_costs():
    cost = 0
    for i in range(1,sites.loc[:,'id'].size+1):
        for j in range(1,clients.loc[:,'id'].size+1):
            mult = x_si[i,j]*demands[j]
            mult1 = mult*productionCosts.loc[0,'productionCenter']
            mult2 = mult*(productionCosts.loc[0,'productionCenter']-productionCosts.loc[0,'automationBonus'])
            mult3 = mult*(productionCosts.loc[0,'productionCenter'])-mult*cost11[i]+mult*productionCosts.loc[0,'distributionCenter']
            cost += mult1*p_s[i]+mult2*a_s[i]+mult3*d_s[i]
    return cost

def building_costs():
    cost = 0
    for i in range(1,sites.loc[:,'id'].size+1):
        cost += buildingCosts.loc[0,'productionCenter']*p_s[i]+(buildingCosts.loc[0,'productionCenter']+buildingCosts.loc[0,'automationPenalty'])*a_s[i]+buildingCosts.loc[0,'distributionCenter']*d_s[i]
    return cost


def rout_costs():
    cost = 0
    for i in range(1,sites.loc[:,'id'].size+1):
        for j in range(1,clients.loc[:,'id'].size+1):
            x_si[i,j]*demands[j]*(routingCosts.loc[0,'secondary']*all_scdistance[i,j]*p_s[i]+
                routingCosts.loc[0,'secondary']*all_scdistance[i,j]*a_s[i]+
                cost22[i]*d_s[i])
    return cost

def slacka(k):
    cost = 0
    for j in range(1,clients.loc[:,'id'].size+1):
        for i in range(1,sites.loc[:,'id'].size+1):
            if i!=k:
                cost += demands[j]*(x_si[k,j]+x_si[i,j]*x_ss[i,k])-capacities.loc[0,'productionCenter']
    return cost

def slackp(k):
    cost = 0
    for j in range(1,clients.loc[:,'id'].size+1):
        for i in range(1,sites.loc[:,'id'].size+1):
            if i!=k:
                cost += demands[j]*(x_si[k,j]+x_si[i,j]*x_ss[i,k])-(capacities.loc[0,'productionCenter']+capacities.loc[0,'automationBonus'])
    return cost

def capacity_costs():
    cost = 0
    for i in range(1,sites.loc[:,'id'].size+1):
        cost += capacities.loc[0,'capacityCost']*(p_s[i]*slackp(i)+a_s[i]*slacka(i))
    return cost

def total_costs():
    return building_costs()+prod_costs()+rout_costs()+capacity_costs()


In [ ]:
mdl.addConstrs(a_s[i]+p_s[i]+d_s[i] >= 0 for i in possible_sites)
mdl.addConstrs(a_s[i]+p_s[i]+d_s[i] <= 1 for i in possible_sites)
mdl.addConstrs(a_s[i]+p_s[i]+d_s[i] >= x_si[i,j] for i in possible_sites for j in all_clients)
mdl.addConstrs(a_s[i]+p_s[i] >= x_ss[i,j] for i in possible_sites for j in possible_sites if i!=j)
mdl.addConstrs(quicksum(x_si[i,j] for i in possible_sites) == 1 for j in all_clients)
mdl.addConstrs(quicksum(x_ss[i,j]*(1-d_s[i])*d_s[j] for i in possible_sites if i!=j) == 1 for j in possible_sites)
mdl.addConstrs(slack_a[i] >= 0 for i in possible_sites)
mdl.addConstrs(slack_p[i] >= 0 for i in possible_sites)
mdl.addConstrs(slack_a[i] >= slacka(j) for i in possible_sites for j in possible_sites)
mdl.addConstrs(slack_p[i] >= slackp(j) for i in possible_sites for j in possible_sites)

mdl.setObjective(total_costs(), GRB.MINIMIZE)

mdl.optimize()